# Customizing EAGO to Solve a Quasiconvex Problem 

[Matthew Wilhelm](https://psor.uconn.edu/person/matthew-wilhelm/)  
Department of Chemical and Biomolecular Engineering, University of Connecticut

[Robert Gottlieb](https://psor.uconn.edu/person/robert-gottlieb/)  
Department of Chemical and Biomolecular Engineering, University of Connecticut

In the following example, we illustrate how EAGO's basic branch-and-bound routine can be adapted for use in other algorithms.

## An Algorithm for Solving Quasiconvex Problems

We'll adapt EAGO to implement the bisection-based algorithm used to solve the quasiconvex optimization problem presented in [1]:

$
\begin{align}
   f^*= \qquad&\min_{\mathbf y\in Y} f(\mathbf y) \\ 
   {\rm s.t.}\;\;&\sum_{i=1}^5 i \cdot y_i - 5 = 0 \\
   &\sum_{i=1}^5 y_i^2 - 0.5\pi \leq 0   \\
   &-\left(\frac{1}{2}y_1^2 + \frac{1}{2}y_2^2 + y_3^2 + 2y_1y_2 + 4y_1y_3 + 2y_2y_3\right)\leq 0   \\ 
   &-y_1^2 - 6y_1y_2 - 2y_2^2 + \text{cos}(y_1) + \pi \leq 0 \\
    &Y = [0, 5]^5
\end{align}
$

where

$
\begin{align}
    f(\mathbf y) = -\frac{\text{ln}((5+y_1)^2 + \sum_{i=1}^5 y_i)}{1 + \sum_{i=1}^{5}y_{i}^{2}}.
\end{align}
$

Interval analysis shows that the objective value is bounded by the interval F such that $f^* \in F=[f^L, f^U] = [-5, 0]$. Introducing an auxiliary variable $t\in T=F$ allows the problem to be formulated as:

$
\begin{align}
    t^*=&\min_{\mathbf y\in Y,t\in T}t\\
    {\rm s.t.} \quad & (24)-(27)\\
    &f(\mathbf y)-t\le 0\\
    &Y=[0,5]^2,\;\;T=[-5,0].
\end{align}
$

Let $\phi_\tau(\mathbf y) = f(\mathbf y) - \tau$ such that $\tau = (t^L + t^U)/2$. We solve for $\mathbf y$ subject to constraints (24)-(27) where $\phi_\tau(\mathbf y) \leq 0$. If this is feasible, $t^* \in [t^L, \tau]$, else $t^* \in [\tau, t^U]$. The interval containing $t^*$ is kept and the other is fathomed. This manner of bisection is repeated until an interval containing a feasible solution with a width of at most $\epsilon$ is located [2].

## Customizing EAGO's script
First, the preprocessing step, upper problem, and postprocessing routines are short-circuited as only a single optimization problem needs to be solved at each iteration.

In [7]:
using EAGO, JuMP
import EAGO: Optimizer, GlobalOptimizer

struct QuasiConvex <: EAGO.ExtensionType end
import EAGO: preprocess!, upper_problem!, postprocess!
function EAGO.preprocess!(t::QuasiConvex, x::GlobalOptimizer)
    x._preprocess_feasibility = true
    end
function EAGO.upper_problem!(t::QuasiConvex, x::GlobalOptimizer)
    x._upper_feasibility = true
    end
function EAGO.postprocess!(t::QuasiConvex, x::GlobalOptimizer)
    x._postprocess_feasibility = true
end

Next, we specify that only an absolute tolerance should be checked for convergence and termination.

In [8]:
import EAGO: convergence_check, termination_check
function EAGO.convergence_check(t::QuasiConvex, x::GlobalOptimizer)
    gap = (x._upper_objective_value - x._lower_objective_value)
    return (gap <= x._parameters.absolute_tolerance)
end
function EAGO.termination_check(t::QuasiConvex, x::GlobalOptimizer)
    flag = EAGO.convergence_check(t, x)
    if flag
        x._end_state = EAGO.GS_OPTIMAL
        x._termination_status_code = MOI.OPTIMAL
        x._result_status_code = MOI.FEASIBLE_POINT
    end
    return flag
end

We then indicate that only the sixth variable, representing $t$, should be branched on. Since we will apply our knowledge about which $t^*$ should be kept in the lower problem definition, we also short-circuit EAGO's repeat_check function here to tell EAGO  not to branch this node, but instead to repeatedly evaluate it.

In [9]:
import EAGO: repeat_check
branch_variable = [i == 6 for i=1:6]
EAGO.repeat_check(t::QuasiConvex, x::GlobalOptimizer) = true

In the lower problem, we then specify that the problem is to be solved locally for a fixed $t$ value. The objective value is then updated and the problem is contracted in order to discard the region which is known to not contain the optimal value.

In [10]:
import EAGO: lower_problem!
function EAGO.lower_problem!(t::QuasiConvex, x::GlobalOptimizer)
    y = x._current_node
    indx = x._sol_to_branch_map[6]
    lower = y.lower_variable_bounds[indx]
    upper = y.upper_variable_bounds[indx]
    midy = (lower + upper)/2.0
    y.lower_variable_bounds[indx] = midy
    y.upper_variable_bounds[indx] = midy
    EAGO.solve_local_nlp!(x)
    feas = x._upper_feasibility
    y.lower_variable_bounds[indx] = feas ?  lower : midy
    y.upper_variable_bounds[indx] = feas ?  midy : upper
    x._lower_objective_value = y.lower_variable_bounds[indx]
    x._upper_objective_value = y.upper_variable_bounds[indx]
    x._lower_feasibility = true
    return
end

We now define the optimizer factory to extend the core EAGO optimizer for this special problem. The SubSolvers() constructor is used to set the extension type (t), as well as the relaxed optimizer (r) and upper-bounding optimizer (u), if necessary. In this case, we will use the default solvers and only set the extension type.

In [13]:
factory = () -> EAGO.Optimizer(SubSolvers(; t = QuasiConvex()));

We now build the JuMP model representing this problem, solve it, and retrieve the solution.

In [12]:
opt = optimizer_with_attributes(factory, 
                                "absolute_tolerance" => 1E-8, 
                                "branch_variable" => branch_variable,
                                "iteration_limit" => 1000,
                                "output_iterations" => 5)
model = Model(opt)
@variable(model, ((i<6) ? 0 : -5) <= y[i=1:6] <= ((i<6) ? 5 : 0))
@constraint(model, sum(i*y[i] for i=1:5) - 5 == 0)
@constraint(model, sum(y[i]^2 for i=1:5) - 0.5*pi^2 <= 0)
@expression(model, expr1, 2*y[1]*y[2] + 4*y[1]*y[3] + 2*y[2]*y[3])
@constraint(model, -(0.5*y[1]^2 + 0.5*y[2]^2 + y[3]^2 + expr1) <= 0)
@expression(model, expr2, log((5 + y[1])^2 + sum(y[i] for i=1:5)))
@constraint(model, -y[1]^2 -6*y[1]*y[2] -2*y[2]^2 +cos(y[1]) + pi <= 0)
@constraint(model, -expr2/(1 + sum(y[i]^2 for i=1:5)) - y[6] <= 0)
@objective(model, Min, y[6])

JuMP.optimize!(model) # Retrieve solution info

---------------------------------------------------------------------------------------------------------------------------------
|  Iteration #  |     Nodes     |  Lower Bound  |  Upper Bound  |      Gap      |     Ratio     |     Timer     |   Time Left   |
---------------------------------------------------------------------------------------------------------------------------------
|             5 |             1 |    -1.719E+00 |    -1.562E+00 |     1.562E-01 |     9.091E-02 |          0.29 |       3599.71 |
|            10 |             1 |    -1.719E+00 |    -1.714E+00 |     4.883E-03 |     2.841E-03 |          0.42 |       3599.58 |
|            15 |             1 |    -1.717E+00 |    -1.717E+00 |     1.526E-04 |     8.887E-05 |          0.61 |       3599.39 |
|            20 |             1 |    -1.717E+00 |    -1.717E+00 |     4.768E-06 |     2.777E-06 |          0.80 |       3599.20 |
|            25 |             1 |    -1.717E+00 |    -1.717E+00 |     1.490E-07 |     8.67

### Reference:
1. C. Jansson, Quasiconvex relaxations based on interval arithmetic, Linear Algebra and its Applications, 324 (2001), pp. 27–53.
2. S. Boyd and L. Vandenberghe, Convex optimization, Cambridge University Press, 2004.